In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error

def bias(x, y): return np.mean(x - y)

from spicy_snow.retrieval import retrieval_from_parameters

In [2]:
lievens_params = [2, 0.5, 0.44]
# wus_params = [2.5, 0.2, 0.55]
# wus_v2_params = [1.5, 0.1, 0.59]

in_dir = Path('~/spicy-snow/SnowEx-Data/').expanduser().resolve()
data_dir = Path('~/scratch/spicy/SnowEx-Data/').expanduser().resolve()

# Create parameter space
A = np.round(np.arange(1, 3.1, 0.1), 2)
B = np.round(np.arange(0, 2.01, 0.1), 2)
C = np.round(np.arange(0, 1.001, 0.01), 2)

def get_stats(x, y):
    if type(x) == xr.DataArray: x = x.values.ravel()
    if type(y) == xr.DataArray: y = y.values.ravel()
    idx = (~np.isnan(x)) & (~np.isnan(y))
    x, y = x[idx], y[idx]
    r, p = pearsonr(x, y)
    b = bias(x, y)
    mae = mean_absolute_error(x, y)
    rmse = mean_squared_error(x, y, squared = False)
    return r, b, mae, rmse

In [3]:
npy_dirs = Path('/bsuhome/zacharykeskinen/scratch/spicy/param_npys')
all_res = xr.open_dataset(npy_dirs.joinpath('grouped.nc'))
print(all_res['pearsonr'].max(['B', 'C']).idxmax('A'))
print(all_res['pearsonr'].max(['C', 'A']).idxmax('B'))
print(all_res['mae'].min(['A', 'B']).idxmin('C'))

<xarray.DataArray 'A' ()>
array(1.5)
<xarray.DataArray 'B' ()>
array(0.)
<xarray.DataArray 'C' ()>
array(0.27)


In [ ]:
res = pd.DataFrame()

for fp in in_dir.glob('*.nc'):
    ds = xr.open_dataset(fp)

    if fp.stem == 'Frasier_2020-02-11':
        r, b, mae, rmse = get_stats(ds['lidar-sd'], ds['snow_depth'].sel(time = '2020-02-16'))
        im_date = pd.to_datetime('2020-02-16')
    else:
        r, b, mae, rmse = get_stats(ds['lidar-sd'], ds['snow_depth'].sel(time = ds.attrs['lidar-flight-time'], method = 'nearest'))
        im_date = pd.to_datetime(ds.sel(time = ds.attrs['lidar-flight-time'], method = 'nearest').time.values.ravel()[0])

    site_name = fp.stem.replace('_', ' ')

    for stat in ['A', 'B', 'C']:

        

        for name, var in zip(['RMSE', 'Pearson R'], [rmse, r]):
            res.loc[site_name, name] = var

    lievens_ds = retrieval_from_parameters(xr.open_dataset(data_dir.joinpath(fp.name)), A = lievens_params[0], B = lievens_params[1], C = lievens_params[2])

    o_r, o_b, o_mae, o_rmse = get_stats(lievens_ds['lidar-sd'], lievens_ds['snow_depth'].sel(time = im_date, method = 'nearest'))

    res.loc[site_name, 'Lievens (2022) rmse'] = o_rmse
    res.loc[site_name, 'Lievens (2022) Pearson R'] = o_r

for name, var in zip(['RMSE', 'bias', 'MAE', 'Pearson R'], [rmse, b, mae, r]):
    res.loc['All Sites', name] = all_res.sel(A = 1.5, B = 0.1, C = 0.27)[name.lower().replace(' ', '')]
res.loc['All Sites', 'Lievens (2022) rmse'] = all_res.sel(A = lievens_params[0], B = lievens_params[1], C = lievens_params[2])['rmse']
res.loc['All Sites', 'Lievens (2022) Pearson R'] = all_res.sel(A = lievens_params[0], B = lievens_params[1], C = lievens_params[2])['pearsonr']